In [110]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import random
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Seed for reproducibility
SEED = 309
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


Using device: cuda


In [111]:
# Dataset paths
dataset_path = 'train_data'
output_path = 'processed_data'
classes = ['tomato', 'cherry', 'strawberry']
target_size = (128, 128)

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)
for cls in classes:
    os.makedirs(os.path.join(output_path, cls), exist_ok=True)

# Blurry image detection function
def is_blurry(image):
    gray_image = np.array(image.convert('L'))
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return laplacian_var > 2000  # Threshold for blurriness

# Preprocess images
for fruit_class in classes:
    folder_path = os.path.join(dataset_path, fruit_class)
    output_folder_path = os.path.join(output_path, fruit_class)
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)

        # Check for blur
        if is_blurry(image):
            print(f"Skipping blurry image: {image_path}")
            continue
        
        # Resize and save preprocessed image
        image = ImageOps.fit(image, target_size, Image.LANCZOS)
        processed_image_path = os.path.join(output_folder_path, image_file)
        image.save(processed_image_path)

print("Preprocessing completed. Processed images are saved in the 'processed_data' folder.")


Skipping blurry image: train_data\tomato\tomato_0055.jpg
Skipping blurry image: train_data\tomato\tomato_0136.jpg
Skipping blurry image: train_data\tomato\tomato_0153.jpg
Skipping blurry image: train_data\tomato\tomato_0156.jpg
Skipping blurry image: train_data\tomato\tomato_0157.jpg
Skipping blurry image: train_data\tomato\tomato_0158.jpg
Skipping blurry image: train_data\tomato\tomato_0228.jpg
Skipping blurry image: train_data\tomato\tomato_0241.jpg
Skipping blurry image: train_data\tomato\tomato_0261.jpg
Skipping blurry image: train_data\tomato\tomato_0307.jpg
Skipping blurry image: train_data\tomato\tomato_0399.jpg
Skipping blurry image: train_data\tomato\tomato_0433.jpg
Skipping blurry image: train_data\tomato\tomato_0500.jpg
Skipping blurry image: train_data\tomato\tomato_0666.jpg
Skipping blurry image: train_data\tomato\tomato_0765.jpg
Skipping blurry image: train_data\tomato\tomato_0793.jpg
Skipping blurry image: train_data\tomato\tomato_0820.jpg
Skipping blurry image: train_da

In [112]:
# Define transformations
transform_train = transforms.Compose([
    transforms.Resize(target_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize(target_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [113]:
# Define data directories
data_root = './train_data/'
preprocessed_data_root = './processed_data/'

# Load datasets with transformations
train_dataset = ImageFolder(root=data_root, transform=transform_train)
test_dataset = ImageFolder(root=preprocessed_data_root, transform=transform_test)

# Split datasets into training and test sets
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

# Load data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Baseline Model

In [114]:
# MLP Model for baseline comparison
class MLPModel(nn.Module):
    def __init__(self, input_size=128*128*3, hidden_size=100, num_classes=3):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the image
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize MLP model, criterion, and optimizer
mlp_model = MLPModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)


In [115]:
def train_mlp_model(data_loader):
    mlp_model.train()
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = mlp_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(data_loader):.4f}")

# Train MLP model
train_mlp_model(train_loader)


Epoch [1/10], Loss: 2.5784
Epoch [2/10], Loss: 1.2750
Epoch [3/10], Loss: 1.1492
Epoch [4/10], Loss: 1.0948
Epoch [5/10], Loss: 1.0803
Epoch [6/10], Loss: 1.0199
Epoch [7/10], Loss: 1.0288
Epoch [8/10], Loss: 0.9715
Epoch [9/10], Loss: 0.9560
Epoch [10/10], Loss: 0.9734


In [116]:
def evaluate_mlp_model(model, train_loader, test_loader):
    model.eval()
    
    def evaluate(data_loader):
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in data_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        return accuracy
    
    train_accuracy = evaluate(train_loader)
    test_accuracy = evaluate(test_loader)
    
    print(f'MLP Model Train Accuracy: {train_accuracy:.2f}%')
    print(f'MLP Model Test Accuracy: {test_accuracy:.2f}%')
    
    return train_accuracy, test_accuracy

# Evaluate MLP Model
evaluate_mlp_model(mlp_model, train_loader, test_loader)


MLP Model Train Accuracy: 57.22%
MLP Model Test Accuracy: 50.28%


(57.21850613154961, 50.27870680044593)

In [117]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 3)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize CNN model
cnn_model = CNNModel().to(device)


In [118]:
# Training function for CNN
def train_cnn_model(data_loader):
    cnn_model.train()
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = cnn_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(data_loader):.4f}")

# Evaluation function for CNN
def evaluate_cnn_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'CNN Model Accuracy: {accuracy:.2f}%')
    return accuracy

# Train and evaluate CNN model
train_cnn_model(train_loader)
evaluate_cnn_model(cnn_model, test_loader)


Epoch [1/10], Loss: 1.1001
Epoch [2/10], Loss: 1.1000
Epoch [3/10], Loss: 1.0997
Epoch [4/10], Loss: 1.1001
Epoch [5/10], Loss: 1.1000
Epoch [6/10], Loss: 1.0997
Epoch [7/10], Loss: 1.1003
Epoch [8/10], Loss: 1.0996
Epoch [9/10], Loss: 1.0997
Epoch [10/10], Loss: 1.1002
CNN Model Accuracy: 36.34%


36.34336677814939